In [1]:
import csv
import random
with open('ClickData4.tsv') as f:
    user=f.readlines()

In [3]:
import datetime
import time
def timeconvert(timestr):
    ifpm=False
    if 'PM' in timestr:
        ifpm=True
    tp=datetime.datetime.strptime(timestr[:-3], "%m/%d/%Y %H:%M:%S")
    tp+=datetime.timedelta(hours=12)
    return tp

In [4]:
userid_dict={}
for i in user:
    tr=i.strip().split('\t')
    userid=tr[0]
    if userid not in  userid_dict:
        userid_dict[userid]=len(userid_dict)

In [7]:
import csv
with open('DocMeta3.tsv') as f:
    data=f.readlines()


In [ ]:
import nltk 
from nltk.tokenize import word_tokenize
news={}
category={}
subcategory={}
for i in data:
    tp=i.strip().split('\t')
    news[tp[1]]=[tp[2],tp[3],word_tokenize(tp[6].lower())]
    category[tp[2]]=0
    subcategory[tp[3]]=0

In [8]:
word_dict0={'PADDING':[0,999999]}

for i in news:
    for j in news[i][2]:
        if j in word_dict0:
            word_dict0[j][1]+=1
        else:
            word_dict0[j]=[len(word_dict0),1]
word_dict={}
for i in word_dict0:
    if word_dict0[i][1]>=2:
        word_dict[i]=[len(word_dict),word_dict0[i][1]]
print(len(word_dict),len(word_dict0))

10000

In [ ]:
import numpy as np
embdict={}
nb=0
import pickle
with open('/home/yourpath/glove.840B.300d.txt','rb')as f:
    linenb=0
    while True:
        j=f.readline()
        if len(j)==0:
            break
        k = j.split()
        word=k[0].decode()
        linenb+=1
        if len(word) != 0:
            tp=[float(x) for x in k[1:]]
            if word in word_dict:
                embdict[word]=tp
                if nb%100==0:
                    print(nb,linenb,word)
                nb+=1

from numpy.linalg import cholesky
word_dict1=word_dict
print(len(embdict),len(word_dict1))
print(len(word_dict1))
lister=[0]*len(word_dict1)
xp=np.zeros(300,dtype='float32')

cand=[]
for i in embdict.keys():
    lister[word_dict1[i][0]]=np.array(embdict[i],dtype='float32')
    cand.append(lister[word_dict1[i][0]])
cand=np.array(cand,dtype='float32')

mu=np.mean(cand, axis=0)
Sigma=np.cov(cand.T)

norm=np.random.multivariate_normal(mu, Sigma, 1)
print(mu.shape,Sigma.shape,norm.shape)

for i in range(len(lister)):
    if type(lister[i])==int:
        lister[i]=np.reshape(norm, 300)
lister[0]=np.zeros(300,dtype='float32')
lister=np.array(lister,dtype='float32')
print(lister.shape)

In [ ]:
news_words=[[0]*30]
news_index={'0':0}
for i in news:
    tp=[]
    news_index[i]=len(news_index)
    for j in news[i][2]:
        if j in word_dict:
            tp.append(word_dict[j][0])
    tp=tp[:30]
    news_words.append(tp+[0]*(30-len(tp)))
    
import numpy as np
news_words=np.array(news_words,dtype='int32') 

In [9]:
import random
def newsample(nnn,ratio):
    if ratio >len(nnn):
        return random.sample(nnn*(ratio//len(nnn)+1),ratio)
    else:
        return random.sample(nnn,ratio)

In [11]:
import itertools
import random
npratio=4
all_train_id=[]
all_train_pn=[]    
all_labeler=[]

all_test_id=[]
all_test_pn=[]    
all_test_labeler=[]
all_test_index=[]

all_user_pos=[]
all_test_user_pos=[]

for raw in user:
    tr=raw.strip().split('\t')
    userid=tr[0]
    if len(tr)==4:
        
        tp=[x.split('#TAB#') for x in tr[2].split('#N#')]
    if len(tr)==3:
        tp=[x.split('#TAB#') for x in tr[2].split('#N#')]

    trainpos=[x[0].split() for x in tp]
    trainneg=[x[1].split() for x in tp]
     
    ppp=list(itertools.chain(*(trainpos)))
    nnn=list(itertools.chain(*(trainneg)))
    #print(ppp)
    
    
    if len(tr)==4:
        tp=[x.split('#TAB#') for x in tr[3].split('#N#')]
        testpos=[x[0].split() for x in tp]
        testneg=[x[1].split() for x in tp]
        
        
        for i in range(len(testpos)):
            tp=[]
            tp.append(len(all_test_pn))
            qqq=list(set(ppp))
            allpos=[int(p) for p in random.sample(qqq,min(50,len(qqq)))[:50]]
            allpos+=[0]*(50-len(allpos))
    
            
            for j in testpos[i]:
                all_test_pn.append(int(j))
                all_test_labeler.append(1)
                all_test_id.append(userid_dict[userid])
                all_test_user_pos.append(allpos)
                
            for j in testneg[i]:
                all_test_pn.append(int(j))
                all_test_labeler.append(0)
                all_test_id.append(userid_dict[userid])
                all_test_user_pos.append(allpos)
            tp.append(len(all_test_pn))
            all_test_index.append(tp)
            
    #nnnnn=list(set(nnn)-set(ppp))

            
    for mp in range(len(trainpos)):
        for ps in trainpos[mp]:
            #tql=list(set(nnn)-set(trainpos[mp]))
            negps=newsample(trainneg[mp],npratio)
            negps.append(ps)
            tplb=[0]*npratio+[1]
            tid=list(range(npratio+1))
            random.shuffle(tid)

            
            yp=[]
            yplb=[]
            for j in tid:
                yp.append(int(negps[j]))
                yplb.append(tplb[j])
            
            qqq=list(set(ppp)-set([ps]))
            allpos=[int(p) for p in random.sample(qqq,min(50,len(qqq)))[:50]]
            allpos+=[0]*(50-len(allpos))
            all_train_pn.append(yp)
            all_labeler.append(yplb)
            all_train_id.append(userid_dict[userid])
            all_user_pos.append(allpos)


In [12]:

def dcg_score(y_true, y_score, k=10):
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order[:k])
    gains = 2 ** y_true - 1
    discounts = np.log2(np.arange(len(y_true)) + 2)
    return np.sum(gains / discounts)


def ndcg_score(y_true, y_score, k=10):
    best = dcg_score(y_true, y_true, k)
    actual = dcg_score(y_true, y_score, k)
    return actual / best


def mrr_score(y_true, y_score):
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order)
    rr_score = y_true / (np.arange(len(y_true)) + 1)
    return np.sum(rr_score) / np.sum(y_true)


In [13]:

def generate_batch_data_random(all_train_pn,all_labeler,all_train_id,batch_size):
    idx = np.arange(len(all_labeler))
    np.random.shuffle(idx)
    y=all_labeler
    batches = [idx[range(batch_size*i, min(len(y), batch_size*(i+1)))] for i in range(len(y)//batch_size+1)]

    while (True):
        for i in batches:
            itx = news_words[all_train_pn[i]]
            tkk=all_user_pos[all_train_id[i]]
            itxx=[itx[:,k,:] for k in range(itx.shape[1])]
            usx=news_words[all_user_pos[i]]
            usxx=[usx[:,k,:] for k in range(usx.shape[1])]
            uid=np.expand_dims(all_train_id[i],axis=1)
            yy=all_labeler[i]
            yield (itxx +usxx+[uid], yy)

In [14]:
def generate_batch_data(all_train_pn,all_labeler,all_train_id,batch_size):
    idx = np.arange(len(all_labeler))
    y=all_labeler
    batches = [idx[range(batch_size*i, min(len(y), batch_size*(i+1)))] for i in range(len(y)//batch_size+1)]

    while (True):
        for i in batches:
            itx = news_words[all_train_pn[i]]
            usx=news_words[all_test_user_pos[i]]
            usxx=[usx[:,k,:] for k in range(usx.shape[1])]
            uid=np.expand_dims(all_train_id[i],axis=1)
            yy=all_labeler[i]
            yield ([itx]+ usxx+[uid], yy)
            
all_train_pn=np.array(all_train_pn,dtype='int32')
all_labeler=np.array(all_labeler,dtype='int32')
all_train_id=np.array(all_train_id,dtype='int32')
all_test_pn=np.array(all_test_pn,dtype='int32')
all_test_labeler=np.array(all_test_labeler,dtype='int32')
all_test_id=np.array(all_test_id,dtype='int32')
all_user_pos=np.array(all_user_pos,dtype='int32')
all_test_user_pos=np.array(all_test_user_pos,dtype='int32')

In [ ]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers import Embedding, concatenate
from keras.layers import Dense, Input, Flatten, average,Lambda

from keras.layers import *
from keras.models import Model, load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint

from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras import initializers #keras2
from keras.utils import plot_model
import numpy as np
from sklearn.metrics import accuracy_score, classification_report
from keras.optimizers import *
import keras
import itertools
import random
results=[]
for npratio in range(4,5):
    
    
    MAX_SENT_LENGTH=30
    MAX_SENTS=50
    
    
    user_id = Input(shape=(1,), dtype='int32')
    
    
 
    
    user_embedding_layer= Embedding(len(user), 50,trainable=True)
    
    user_embedding= user_embedding_layer(user_id)
    
    user_embedding0= Dense(200,activation='relu')(user_embedding)
    user_embedding0= Flatten()(user_embedding0)
    
    user_embedding1= Dense(200,activation='relu')(user_embedding)
    user_embedding1= Flatten()(user_embedding1)
    
    
    #user_embedding= RepeatVector(1+npratio)(user_embedding0)
    

    
    
    sentence_input = Input(shape=(MAX_SENT_LENGTH,), dtype='int32')
    
    
    embedding_layer = Embedding(33050 , 300, weights=[lister],trainable=True)
    
    embedded_sequences = embedding_layer(sentence_input)
    d_et=Dropout(0.2)(embedded_sequences)
    
    l_cnnt = Convolution1D(nb_filter=400, filter_length=3,  padding='same', activation='relu', strides=1)(d_et)
    d_ct=Dropout(0.2)(l_cnnt)
    
    attention = Dense(200,activation='tanh')(d_ct)
    attention = Dot((2, 1))([attention, user_embedding0])
    attention_weight = Activation('softmax')(attention)
    l_attt=keras.layers.Dot((1, 1))([d_ct, attention_weight])
    
    
    sentEncodert = Model([sentence_input,user_id], l_attt)
    
    
    review_input = [keras.Input((MAX_SENT_LENGTH,), dtype='int32') for _ in range(MAX_SENTS)]
    candidate_vecssent = [sentEncodert([review,user_id]) for review in review_input]
    candidate_vecssent2 =concatenate([Lambda(lambda x: K.expand_dims(x,axis=1))(review) for review in candidate_vecssent],axis=1)
    #review_input = Input(shape=(MAX_SENTS, MAX_SENT_LENGTH), dtype='int32')
    #review_encoder = TimeDistributed(sentEncodert)(review_input)
    #print(K.int_shape(candidate_vecssent2))
    
    
    attention2 = Dense(200,activation='tanh')(candidate_vecssent2)
    attention2 = keras.layers.Dot((2, 1))([attention2, user_embedding1])
    attention_weight2 = Activation('softmax')(attention2)
    l_att2=keras.layers.Dot((1, 1))([candidate_vecssent2, attention_weight2])
    #l_att2=AttLayer()(candidate_vecssent2)
    
    candidates = [keras.Input((MAX_SENT_LENGTH,), dtype='int32') for _ in range(1+npratio)]
    candidate_vecs = [sentEncodert([candidate,user_id]) for candidate in candidates]
    
    
    logits = [keras.layers.dot([l_att2, candidate_vec], axes=-1) for candidate_vec in candidate_vecs]
    logits = keras.layers.Activation(keras.activations.softmax)(keras.layers.concatenate(logits))
    
    
    model = Model(candidates+review_input+[user_id], logits)
    
    
    candidate_one = keras.Input((MAX_SENT_LENGTH,))
    candidate_one_vec = sentEncodert([candidate_one,user_id])
    score = keras.layers.Activation(keras.activations.sigmoid)(
        keras.layers.dot([l_att2, candidate_one_vec], axes=-1))
    model2 = keras.Model([candidate_one]+review_input+[user_id], score)
    
    
    model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['acc'])

    for ep in range(3):
        traingen=generate_batch_data_random(all_train_pn,all_labeler,all_train_id, 100)
        
        model.fit_generator(traingen, epochs=1,steps_per_epoch=len(all_train_id)//100)
        #for ep in range(1):    
        testgen=generate_batch_data(all_test_pn,all_test_labeler,all_test_id, 100)
        cr = model2.predict_generator(testgen, steps=len(all_test_id)//100,verbose=1)
        from sklearn.metrics import roc_auc_score
        all_auc=[]
        all_mrr=[]
        all_ndcg=[]
        all_ndcg2=[]
        for m in all_test_index:
            if np.sum(all_test_labeler[m[0]:m[1]])!=0 and m[1]<len(cr):
        
                all_auc.append(roc_auc_score(all_test_labeler[m[0]:m[1]],cr[m[0]:m[1],0]))
                all_mrr.append(mrr_score(all_test_labeler[m[0]:m[1]],cr[m[0]:m[1],0]))
                all_ndcg.append(ndcg_score(all_test_labeler[m[0]:m[1]],cr[m[0]:m[1],0],k=5))
                all_ndcg2.append(ndcg_score(all_test_labeler[m[0]:m[1]],cr[m[0]:m[1],0],k=10))
        results.append([np.mean(all_auc),np.mean(all_mrr),np.mean(all_ndcg),np.mean(all_ndcg2)])
        print(np.mean(all_auc),np.mean(all_mrr),np.mean(all_ndcg),np.mean(all_ndcg2))